This notebook is part of the final examination of the **Python for Genomic Data Science** course on Coursera

The goal is to read in a multi-`FASTA` file and find reading frames and repeating subsequences. The assignment is simplified by the fact that no gaps occur in the data. Also, we are concerned with the forward (5') direction only.

The tools and algorithms in this notebook are built upon Biopython.

In [43]:
import os
from pathlib import Path

import numpy as np

# This is a header module with methods designed to operate on strings and Bio.SeqIO.SeqRecord
import utils
# This is a specialized UserList class that holds the SeqRecords.
from sequencerecords import SequenceRecordList

# Read the Data

In [44]:
input_file = Path("dna2.fasta")
assert input_file.exists()

# The list of SeqRecords is sorted from shortest to longest sequence length
sequences = SequenceRecordList.read_file(os.fspath(input_file), "fasta")

In [45]:
print(
    f"There are {len(sequences)} sequences in the file. The shortest is {sequences[0].id}"
    f" with {len(sequences[0].seq)} characters and the longest is {sequences[-1].id} with"
    f" {len(sequences[-1].seq)} characters"
)

There are 18 sequences in the file. The shortest is gi|142022655|gb|EQ086233.1|346 with 115 characters and the longest is gi|142022655|gb|EQ086233.1|255 with 4894 characters


# Question
*How many records are in the multi-FASTA file?*

The answer is **18**

# Question
*What is the length of the longest sequence in the file?*

The answer is **4894**

# Question
*What is the length of the shortest sequence in the file?*

The answer is **115**

# Question:

*What is the length of the longest ORF appearing in reading frame 2 of any of the sequences?*

## Step A. Finding the Open Reading Frames (ORFs)

In [46]:
sequences_rf_tables = {
    seq_rec.id : utils.get_forward_rf_tables(seq_rec)
    for seq_rec in sequences
}

In [5]:
# Let's verify one of the RF tables. **NOTE** These are NOT the ORF
header = f"Sequence ID: {sequences[0].id}"
seperator = "*" * len(header)

print(seperator)
print(header)
print(seperator)
print(str(sequences[0].seq))
print(seperator)
for rf_table_id, rf_table in sequences_rf_tables[sequences[0].id].items():
    print(f"Table {rf_table_id}: {str(rf_table)}")


*******************************************
Sequence ID: gi|142022655|gb|EQ086233.1|346
*******************************************
GCGGTCCCGGCGCCGCAGGCCGTCCGGCTCCTGCAGCGCGCCGAACCGGGTCTCGCGGTGATTGCCCAGCGTACCGAGATGCGCCCGGCCTGGGCCGTGATGGCGCAGTGCGGCC
*******************************************
Table 1: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 115]
Table 2: [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115]
Table 3: [0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 115]


## Step B. Get all ORFs

In [6]:
sequences_orfs = {
    seq_rec.id : utils.get_all_forward_orf(seq_rec, sequences_rf_tables[seq_rec.id])
    for seq_rec in sequences
}

In [9]:
# Lets verify forward ORFs
print(sequences_orfs[sequences[0].id])

{1: [], 2: [], 3: []}

In [12]:
# Turns out the shortest sequence doesn't have any ORFs. What about the next one?
print(sequences_orfs[sequences[1].id])

print(sequences[1][274: 430])

{1: [], 2: [(274, 430), (349, 430), (379, 430)], 3: [(89, 278), (311, 353)]}
ID: gi|142022655|gb|EQ086233.1|322
Name: gi|142022655|gb|EQ086233.1|322
Description: gi|142022655|gb|EQ086233.1|322 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
Number of features: 0
Seq('ATGACCGCGCAACGGGCGAGCGAGCGGCAGTACCTGAATGCGGAGCTCGATTCG...TGA')


Success! The first codon is a START and last is a STOP

In [18]:
target_rf = 2
longest_orf_sequences = dict.fromkeys(sequences_orfs.keys(), 0)
for seq_rec_id, forward_orf_tables in sequences_orfs.items():
    forward_orf_table = forward_orf_tables[target_rf]
    sub_seq_lengths_pairs = [
        np.diff(index_pair)[0]
        for index_pair in forward_orf_table
    ]
    if len(sub_seq_lengths_pairs) > 0:
        longest_orf_sequences[seq_rec_id] = np.max(sub_seq_lengths_pairs)

print(f"All ORFs in RF {target_rf}")
print(longest_orf_sequences)

print(f"The longest ORF in RF {target_rf} is {np.max(list(longest_orf_sequences.values()))}")

All ORFs in RF 2
{'gi|142022655|gb|EQ086233.1|346': 0, 'gi|142022655|gb|EQ086233.1|322': 156, 'gi|142022655|gb|EQ086233.1|88': 138, 'gi|142022655|gb|EQ086233.1|584': 27, 'gi|142022655|gb|EQ086233.1|594': 33, 'gi|142022655|gb|EQ086233.1|304': 0, 'gi|142022655|gb|EQ086233.1|75': 504, 'gi|142022655|gb|EQ086233.1|277': 279, 'gi|142022655|gb|EQ086233.1|4': 126, 'gi|142022655|gb|EQ086233.1|527': 570, 'gi|142022655|gb|EQ086233.1|250': 552, 'gi|142022655|gb|EQ086233.1|45': 528, 'gi|142022655|gb|EQ086233.1|396': 1281, 'gi|142022655|gb|EQ086233.1|293': 420, 'gi|142022655|gb|EQ086233.1|454': 822, 'gi|142022655|gb|EQ086233.1|91': 237, 'gi|142022655|gb|EQ086233.1|16': 1458, 'gi|142022655|gb|EQ086233.1|255': 1185}
The longest ORF in RF 2 is 1458


The answer is **1458**

# Question:

*What is the starting position of the longest ORF in reading frame 3 in any of the sequences? The position should indicate the character number where the ORF begins. For instance, the following ORF:*
```text
> sequence1
ATGCCCTAG
```
*starts at position 1.*

Fortunately, I can reuse some machinery from the previous question to help with this one

In [29]:
target_rf = 3
longest_orf_positions = dict.fromkeys(sequences_orfs.keys(), (0, -1))
for seq_rec_id, forward_orf_tables in sequences_orfs.items():
    forward_orf_table = forward_orf_tables[target_rf]
    sub_seq_lengths = [
        np.diff(index_pair)[0]
        for index_pair in forward_orf_table
    ]
    if len(sub_seq_lengths) > 0:
        longest_orf_argmax = np.argmax(sub_seq_lengths)
        index_pair = forward_orf_table[longest_orf_argmax]
        position_pair = utils.convert_index_pair_to_position_pair(index_pair)
        longest_orf_positions[seq_rec_id] = position_pair

print(f"All ORFs in RF {target_rf}")
print(longest_orf_positions)

longest_orf_by_length = sorted(
    list(longest_orf_positions.items()),
    key=lambda ident_position_pair: np.diff(ident_position_pair[1])[0],
    reverse=True
)
print(f"The longest ORF in RF {target_rf} is {longest_orf_by_length[0]} of length {np.diff(longest_orf_by_length[0][1])[0]}")

All ORFs in RF 3
{'gi|142022655|gb|EQ086233.1|346': (0, -1), 'gi|142022655|gb|EQ086233.1|322': (90, 279), 'gi|142022655|gb|EQ086233.1|88': (0, -1), 'gi|142022655|gb|EQ086233.1|584': (348, 480), 'gi|142022655|gb|EQ086233.1|594': (66, 279), 'gi|142022655|gb|EQ086233.1|304': (621, 768), 'gi|142022655|gb|EQ086233.1|75': (57, 261), 'gi|142022655|gb|EQ086233.1|277': (759, 930), 'gi|142022655|gb|EQ086233.1|4': (1278, 1497), 'gi|142022655|gb|EQ086233.1|527': (636, 2457), 'gi|142022655|gb|EQ086233.1|250': (1374, 1623), 'gi|142022655|gb|EQ086233.1|45': (699, 1023), 'gi|142022655|gb|EQ086233.1|396': (429, 654), 'gi|142022655|gb|EQ086233.1|293': (2334, 3045), 'gi|142022655|gb|EQ086233.1|454': (3096, 4497), 'gi|142022655|gb|EQ086233.1|91': (2856, 3444), 'gi|142022655|gb|EQ086233.1|16': (1440, 3084), 'gi|142022655|gb|EQ086233.1|255': (1641, 1926)}
The longest ORF in RF 3 is ('gi|142022655|gb|EQ086233.1|527', (636, 2457)) of length 1821


The answer is position **636**

# Question
*What is the length of the longest ORF appearing in any sequence and in any forward reading frame?*

Since we have already answered for RF 2 and RF 3, let's also do RF 1

In [30]:
target_rf = 1
longest_orf_sequences = dict.fromkeys(sequences_orfs.keys(), 0)
for seq_rec_id, forward_orf_tables in sequences_orfs.items():
    forward_orf_table = forward_orf_tables[target_rf]
    sub_seq_lengths_pairs = [
        np.diff(index_pair)[0]
        for index_pair in forward_orf_table
    ]
    if len(sub_seq_lengths_pairs) > 0:
        longest_orf_sequences[seq_rec_id] = np.max(sub_seq_lengths_pairs)

print(f"All ORFs in RF {target_rf}")
print(longest_orf_sequences)

print(f"The longest ORF in RF {target_rf} is {np.max(list(longest_orf_sequences.values()))}")

All ORFs in RF 1
{'gi|142022655|gb|EQ086233.1|346': 0, 'gi|142022655|gb|EQ086233.1|322': 0, 'gi|142022655|gb|EQ086233.1|88': 120, 'gi|142022655|gb|EQ086233.1|584': 90, 'gi|142022655|gb|EQ086233.1|594': 42, 'gi|142022655|gb|EQ086233.1|304': 105, 'gi|142022655|gb|EQ086233.1|75': 180, 'gi|142022655|gb|EQ086233.1|277': 204, 'gi|142022655|gb|EQ086233.1|4': 249, 'gi|142022655|gb|EQ086233.1|527': 195, 'gi|142022655|gb|EQ086233.1|250': 1560, 'gi|142022655|gb|EQ086233.1|45': 2394, 'gi|142022655|gb|EQ086233.1|396': 1059, 'gi|142022655|gb|EQ086233.1|293': 312, 'gi|142022655|gb|EQ086233.1|454': 1044, 'gi|142022655|gb|EQ086233.1|91': 1296, 'gi|142022655|gb|EQ086233.1|16': 1509, 'gi|142022655|gb|EQ086233.1|255': 1443}
The longest ORF in RF 1 is 2394


So for RF 1, 2, and 3 the longest ORFs are 2394, 1458, and 1821. So RF 1 has the longest ORF.

The answer is **2394**

# Question
*What is the length of the longest forward ORF that appears in the sequence with the identifier  gi|142022655|gb|EQ086233.1|16?*

According to the blocks above
 * RF 1: 1509
 * RF 2: 1458
 * RF 3: 3084 - 1440 = 1644

So the answer is **1644**

# Question
*Find the most frequently occurring repeat of length 6 in all sequences. How many times does it occur in all?*

In [31]:
all_possible_subseq = sequences.get_all_forward_repeating_overlapping_subsequences_of_length(6)

In [34]:
subseq_count_list = sorted(
    [
        (key, value["count"])
        for key, value in all_possible_subseq.items()
    ],
    key=lambda id_count: id_count[1],
    reverse=True
)

In [36]:
print(f"The subseq {subseq_count_list[0][0]} occurs the most with {subseq_count_list[0][1]} occurrences")

The subseq GCGCGC occurs the most with 153 occurrences


The answer is **153**

# Question
*Find all repeats of length 12 in the input file. Let's use Max to specify the number of copies of the most frequent repeat of length 12.  How many different 12-base sequences occur Max times?*

In [37]:
all_possible_subseq = sequences.get_all_forward_repeating_overlapping_subsequences_of_length(12)
subseq_count_list = sorted(
    [
        (key, value["count"])
        for key, value in all_possible_subseq.items()
    ],
    key=lambda id_count: id_count[1],
    reverse=True
)

4


In [38]:
Max_counts = sum(
    [
        True
        for (_, counter) in subseq_count_list
        if counter == subseq_count_list[0][1]
    ]
)

print(f"The maximum number length 12 subsequences occurs {Max_counts} times")

The maximum number length 12 subsequences occurs 4 times


The answer is **4**

# Question
*Which one of the following repeats of length 7 has a maximum number of occurrences?*

In [39]:
all_possible_subseq = sequences.get_all_forward_repeating_overlapping_subsequences_of_length(7)
subseq_count_list = sorted(
    [
        (key, value["count"])
        for key, value in all_possible_subseq.items()
    ],
    key=lambda id_count: id_count[1],
    reverse=True
)

Let's examine the top 5 entries

In [42]:
subseq_count_list[0:5]

[('CGCGCCG', 63),
 ('CGCCGCG', 62),
 ('GCCGCGC', 61),
 ('GCGCGCG', 59),
 ('GCGCGGC', 58)]

So one maximum number of forward repeating subsequences exist, which is CGCGCCG.

The answer is **CGCGCCG**